# Data extraction and cleaning

In [53]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
load_dotenv()

True

## Used car and related data

Data sources:
- Used car data: [Kaggle](https://www.kaggle.com/datasets/shubham1kumar/usedcar-data?select=UserCarData.csv)
- GDP data: [Federal Reserve Bank of Philadelphia](https://www.philadelphiafed.org/surveys-and-data/real-time-data-research/gdpplus)
- Global Supply Chain Pressure Index: [Federal Reserve Bank of New York](https://www.newyorkfed.org/research/policy/gscpi#/overview)
- Inflation: [Federal Reserve Bank of St. Louis](https://www.stlouisfed.org/research/economists/mccracken/fred-databases)
- Used Vehcile Value Index: [Manheim](https://site.manheim.com/en/services/consulting/used-vehicle-value-index.html)
- VIN information: [Wikibooks](https://en.wikibooks.org/wiki/Vehicle_Identification_Numbers_(VIN_codes)/World_Manufacturer_Identifier_(WMI)) & [Wikibooks](https://en.wikibooks.org/wiki/Vehicle_Identification_Numbers_(VIN_codes)/Model_year)

In [95]:
df_uc = pd.read_csv('raw/vehicles.csv', parse_dates=['posting_date'])
df_gdp = pd.read_excel('raw/gdpplus.xlsx')
df_gscpi = pd.read_excel('raw/gscpi_data.xlsx', sheet_name='GSCPI Monthly Data', names=['date', 'gscpi'])
df_inf = pd.read_csv('raw/2024-12.csv', usecols=['sasdate', 'CPIAUCSL'], skiprows=range(1, 3))
df_uvvi = pd.read_excel('raw/December-2024-Manheim-Used-Vehicle-Value-Index.xlsx', sheet_name='DATA', usecols='A,F', names=['date', 'uvvi'])
df_manufacturers = pd.read_excel('raw/manufacturers.xlsx')

In [4]:
df_uc.sample(3)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
189623,7302682076,https://southcoast.craigslist.org/ctd/d/boston...,south coast,https://southcoast.craigslist.org,42590,2011.0,NaN,Maserati GranTurismo Coupe 2D,good,NaN,...,NaN,coupe,black,https://images.craigslist.org/01313_fuA5hKpFAI...,Carvana is the safer way to buy a car During t...,NaN,ma,42.350000,-71.060000,2021-04-06 13:21:37-04:00
37857,7312131911,https://modesto.craigslist.org/ctd/d/keyes-201...,modesto,https://modesto.craigslist.org,22995,2016.0,ford,f150 xlt,NaN,NaN,...,NaN,NaN,NaN,https://images.craigslist.org/00R0R_adAnkwSx3s...,"AUTOMATIC 4WD V6,2.7LITER 177,959 MILES BACK U...",NaN,ca,37.596846,-120.938695,2021-04-24 16:07:43-07:00
54484,7316751275,https://sandiego.craigslist.org/csd/cto/d/san-...,san diego,https://sandiego.craigslist.org,5799,2015.0,nissan,sentra sv sedan 4d,good,4 cylinders,...,compact,sedan,white,https://images.craigslist.org/01515_gcTpHOj2iT...,"2015 Nissan SUV with 96,000 miles clean interi...",NaN,ca,32.811900,-117.067400,2021-05-04 08:54:13-07:00


In [5]:
df_uc.drop(columns=['url', 'region_url', 'image_url', 'description', 'lat', 'long', 'title_status', 'size', 'county'], inplace=True)

In [6]:
df_uc = df_uc.loc[df_uc['posting_date'].notnull()]

In [7]:
df_uc['posting_date'] = pd.to_datetime(df_uc['posting_date'], utc=True).dt.date

In [8]:
df_uc.sample(3)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,VIN,drive,type,paint_color,state,posting_date
312095,7312830072,bend,84747,2019.0,ford,super duty f-350 drw king,NaN,8 cylinders,diesel,9130.0,automatic,1FT8W3DT4KEE48612,4wd,NaN,NaN,or,2021-04-26
52172,7316488207,sacramento,36590,2016.0,audi,a7 prestige sedan 4d,good,6 cylinders,gas,48240.0,other,WAU2GAFC0GN016950,NaN,sedan,black,ca,2021-05-03
217572,7313255411,st cloud,39900,2015.0,ford,f-450 xlt,NaN,NaN,diesel,79000.0,automatic,NaN,NaN,NaN,NaN,mn,2021-04-27


In [ ]:
df_uc.to_csv('used_car.csv', index=False)

In [96]:
df_gdp.tail(3)

,OBS_YEAR,OBS_QUARTER,OBS_QUARTER2,RECBARS,GRGDP_DATA,GRGDI_DATA,GDPPLUS_DATA
256,2024,1,0.00,0,1.61516,2.90865,2.98976
257,2024,2,0.25,0,2.94573,1.99909,2.38250
258,2024,3,0.50,0,2.79423,2.20132,2.38462


In [97]:
df_gdp['year_quarter'] = df_gdp['OBS_YEAR'].astype(str) + '-Q' + df_gdp['OBS_QUARTER'].astype(str)

In [98]:
df_gdp = df_gdp[['year_quarter', 'GRGDP_DATA']].rename(columns={'GRGDP_DATA': 'gdp_growth'})

In [99]:
df_gdp['gdp_growth'] = df_gdp.gdp_growth/100

In [100]:
df_gscpi.head(3)

,date,gscpi
0,28-Feb-1998,-0.435580
1,31-Mar-1998,-0.060024
2,30-Apr-1998,-0.118535


In [101]:
df_gscpi['date'] = pd.to_datetime(df_gscpi.date)

In [102]:
df_gscpi['year'] = df_gscpi.date.apply(lambda x: x.year)
df_gscpi['quarter'] = df_gscpi.date.apply(lambda x: x.quarter)
df_gscpi['year_quarter'] = df_gscpi['year'].astype(str) + '-Q' + df_gscpi['quarter'].astype(str)

In [103]:
df_gscpi = df_gscpi.groupby('year_quarter').agg({'gscpi': 'mean'}).reset_index()

In [104]:
df_inf.head(3)

,sasdate,CPIAUCSL
0,3/1/1959,28.9933
1,6/1/1959,29.0433
2,9/1/1959,29.1933


In [105]:
df_inf['date'] = pd.to_datetime(df_inf['sasdate'], format='%m/%d/%Y')

In [106]:
df_inf.drop(columns=['sasdate'], inplace=True)

In [107]:
df_inf['year'] = df_inf.date.apply(lambda x: x.year)
df_inf['quarter'] = df_inf.date.apply(lambda x: x.quarter)
df_inf['year_quarter'] = df_inf['year'].astype(str) + '-Q' + df_inf['quarter'].astype(str)

In [108]:
df_inf.drop(columns=['year', 'quarter', 'date'], inplace=True)

In [109]:
df_inf['inflation_rate'] = df_inf['CPIAUCSL'].apply(lambda x: np.log(x)).diff()

In [118]:
df_inf.drop(columns=['CPIAUCSL'], inplace=True)

In [110]:
df_uvvi.head(3)

,date,uvvi
0,1997-01-01,NaN
1,1997-02-01,0.003280
2,1997-03-01,-0.001743


In [111]:
df_uvvi['year'] = df_uvvi.date.apply(lambda x: x.year)
df_uvvi['quarter'] = df_uvvi.date.apply(lambda x: x.quarter)
df_uvvi['year_quarter'] = df_uvvi['year'].astype(str) + '-Q' + df_uvvi['quarter'].astype(str)

In [112]:
df_uvvi = df_uvvi.groupby('year_quarter').agg({'uvvi': 'mean'}).reset_index()

In [119]:
df_ts = df_uvvi.merge(df_gscpi, on='year_quarter').merge(df_gdp, on='year_quarter').merge(df_inf, on='year_quarter')

In [120]:
df_ts['date'] = pd.to_datetime(df_ts.year_quarter)

C:\Users\julia\AppData\Local\Temp\ipykernel_20280\2194566706.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ts['date'] = pd.to_datetime(df_ts.year_quarter)


In [121]:
df_ts.drop(columns=['year_quarter'], inplace=True)

In [122]:
df_ts.to_csv('time_series.csv', index=False)

In [7]:
cohere_api_key = os.getenv('API_KEY_COHERE')

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere
from pydantic import BaseModel, Field,  ValidationError, field_validator
from typing import Optional


llm = ChatCohere(temperature=0.0, cohere_api_key=cohere_api_key)

# Define your desired data structure.
class Manufacturer(BaseModel):
    user_input: str = Field(description="The user input that contains the car manufacturer name.")
    manufacturer: Optional[str] = Field(
        description="This is the short lowercased name of the manufacturer retrieved from the user input.")
    
    @field_validator('manufacturer')
    def validate_manufacturer(cls, v):
        if v and len(v) < 3:
            raise ValueError('Manufacturer name must be at least 3 characters long.')
        return v

# Set up a parser + inject instructions into the prompt template.
structured_llm = llm.with_structured_output(Manufacturer)


system = """You will be given a series of car manufacturer names from a user.
Often the input includes additional 
information such as the location where the car was manufactured,
the body style, the model or also in which country the manufacturer is located.
Use your world knowledge to only retrieve the short 
and crisp name of the car manufacturer in lowercase letters 
and dash seperated if the name consists of more than one word.

Here are some examples of user inputs and the expected manufacturer names:

example_input: "Mercedes Benz truck & bus (Argentina)"
example_output: {{"user_input": "Mercedes Benz truck & bus (Argentina)", "manufacturer": "mercedes-benz"}}

example_input: "'Toyota Motor Europe (based in Belgium) used for Toyota ProAce, Toyota ProAce City and Toyota ProAce Max made by PSA/Stellantis'"
example_output: {{"user_input": "'Toyota Motor Europe (based in Belgium) used for Toyota ProAce, Toyota ProAce City and Toyota ProAce Max made by PSA/Stellantis'", "manufacturer": "toyota"}}

example_input: "Tesla, Inc. (US-built MPVs (e.g. Model X, Model Y))"
example_output: {{"user_input": "Tesla, Inc. (US-built MPVs (e.g. Model X, Model Y))", "manufacturer": "tesla"}}
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm

In [26]:
results = []

In [34]:
wmis = df_manufacturers.wmi.values.tolist()
for wmi in tqdm(wmis[377+190:]):
    user_input = df_manufacturers.loc[df_manufacturers.wmi == wmi, 'manufacturer_long'].values[0]
    try:
        output = few_shot_structured_llm.invoke(user_input).model_dump()
        output['wmi'] = wmi
        results.append(output)
    except ValidationError as e:
        print(f"{wmi}: {e}") 

  0%|          | 0/1746 [00:00<?, ?it/s]


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [37]:
df_res = pd.DataFrame(results)

In [44]:
df_result = df_res[['wmi', 'manufacturer']]

In [53]:
df_result['manufacturer'] = df_result['manufacturer'].apply(lambda x: x.lower())

C:\Users\julia\AppData\Local\Temp\ipykernel_28012\3756598195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['manufacturer'] = df_result['manufacturer'].apply(lambda x: x.lower())


In [56]:
df_result.set_index('wmi')['manufacturer'].to_dict()

{'AAA': 'audi',
 'AAK': 'faw',
 'AAM': 'man',
 'AAP': '',
 'AAV': 'volkswagen',
 'AAW': 'challenger-trailer',
 'AA9': 'tr-tec',
 'CN1': 'tr-tec',
 'ABJ': 'mitsubishi',
 'ABM': 'bmw',
 'ACV': 'isuzu',
 'AC5': 'hyundai',
 'ADB': 'mercedes-benz',
 'ADD': '',
 'ADM': 'general-motors',
 'ADN': 'nissan',
 'ADR': 'renault',
 'ADX': 'tata',
 'AFA': '',
 'AFB': 'mazda',
 'AFD': 'baic',
 'AHH': 'hino',
 'AHM': 'mercedes-benz',
 'AHT': 'toyota',
 'BF9/': 'kibo',
 'BUK': 'kiira-motors-corporation',
 'BR1': 'mercedes-benz',
 'EBZ': 'nizhekotrans',
 'DF9/': 'laraki',
 'HA0': 'wuxi-sundiro-electric-vehicle-co-ltd',
 'HA6': 'niu technologies',
 'HA7': 'jinan-qingqi-kr-motors-co-ltd',
 'HES': 'smart',
 'HGL': 'farizon-auto',
 'HGX': 'wuling',
 'HJR': 'jetour',
 'HL4': 'morini',
 'HRV': 'beijing-henrey',
 'HZ2': 'taizhou-zhilong-technology-co-ltd',
 'H0D': 'taizhou-qianxin-vehicle-co-ltd',
 'JAA': 'isuzu',
 'JAB': 'isuzu',
 'JAC': 'isuzu',
 'JAE': 'acura',
 'JAL': 'isuzu',
 'JAM': 'isuzu',
 'JA3': 'mits

In [55]:
df_result

,wmi,manufacturer
0,AAA,audi
1,AAK,faw
2,AAM,man
3,AAP,
4,AAV,volkswagen
...,...,...
697,MM8,mazda
698,MNA,ford
699,MNB,ford
700,MNC,ford


## Insurance Claims data

Data source:

- Claims descriptions: [GitHub](https://github.com/Mahesh3394/Claim-Description-Classification)

In [11]:
df_claims = pd.read_excel('raw/Dataset_Public.xlsx')

In [12]:
df_claims.head(3)

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
1,CLAIMANT ALLEGES SHE SUFFERED INJURIES IN AN E...,GB,Elevator/Escalator
2,"IV PASSENGER SUSTAINED INJURIES, OV AND IV COL...",AB,Sideswipe or lane change


In [13]:
df_claims = df_claims.drop(columns=['Coverage Code', 'Accident Source']).rename(columns={'Claim Description': 'claim_description'})

In [14]:
df_claims.to_csv('claims.csv', index=False)

## Anual reports

In [15]:
from bs4 import BeautifulSoup as soup
import requests as r
import pandas
import time
import os
import datetime
import random
import shutil
from string import punctuation